# Anti-Money Laundering and Fraud Prediction
This is a breakdown of the overall models that can be developed to make AI models to predict and detect money-laundering or fraud within financial datasets.
Data sources for these datasets come from sources on Kaggle:
- [Credit Card Fraud Detection | Kaggle](https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud?datasetId=310&sortBy=voteCount)
- [Fake Bills | Kaggle]("https://www.kaggle.com/datasets/alexandrepetit881234/fake-bills")

These data sets contain various amounts of data dating till 2013, with various levels of information that is captured from financial entities.

## Packages to install for below
Make sure pip is up to date for these packages to install
```
python.exe -m pip install --upgrade pip
```

To install the SciKit (sklearn) packages use the below command:
```
pip install scikit-learn
```

To install Seaborn packages use the below command:
```
pip install seaborn
```

To install the Plotly packages use the below command:
```
pip install plotly
```

## Library Imports
To start the overall work click play on the play button for the packages

In [39]:
import numpy as np # linear algebra breakdown
import pandas as pd # data processing, CSV files input/output
import matplotlib.pyplot as plt # graph plotting
import seaborn as sns 
import warnings

import plotly.express as px
import plotly.graph_objects as go

from numpy import percentile
from mpl_toolkits.mplot3d import Axes3D

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import average_precision_score


warnings.filterwarnings('ignore')

%matplotlib inline

## Dataset files

From the above components you will then be able to import the different files that are needing to be analysed with Pandas.
Pandas will be able to pull in the different files, for example with this work from Github.

In [40]:
#urls = ["https://github.com/jono120/fictional-octo-potato/raw/main/transaction_data/fake_bills.csv", "https://github.com/Jono120/fictional-octo-potato/tree/main/transaction_data/bank.csv", "https://github.com/Jono120/fictional-octo-potato/tree/main/transaction_data/train_trd.csv"]
#df = pd.read_csv("https://github.com/jono120/fictional-octo-potato/raw/main/transaction_data/fake_bills.csv")
#df = pd.read_csv("https://github.com/Jono120/fictional-octo-potato/tree/main/transaction_data/bank.csv")
#df = pd.read_csv("https://github.com/Jono120/fictional-octo-potato/tree/main/transaction_data/train_trd.csv")

#dfs = [pd.read_csv(url) for url in urls]
#df = pd.concat(dfs)
#print(df.head())
#df.info()

In [41]:
df_full = pd.read_csv("https://github.com/jono120/fictional-octo-potato/raw/main/transaction_data/fake_bills.csv", sep = ';')

df_false = df_full.loc[df_full['is_genuine']==False]
df_true = df_full.loc[df_full['is_genuine']==True]
df_true = df_true.fillna(df_true.median())
df_false = df_false.fillna(df_false.median())

In [42]:
print("Shape of dataset:", df_full.shape)
print("Overview of the data:" )
print(df.head())
df_full.info()


Shape of dataset: (1500, 7)
Overview of the data:
  is_genuine;diagonal;height_left;height_right;margin_low;margin_up;length
0         True;171.81;104.86;104.95;4.52;2.89;112.83                      
1         True;171.46;103.36;103.66;3.77;2.99;113.09                      
2           True;172.69;104.48;103.5;4.4;2.94;113.16                      
3         True;171.36;103.91;103.94;3.62;3.01;113.51                      
4         True;171.73;104.28;103.46;4.04;3.48;112.54                      
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   is_genuine    1500 non-null   bool   
 1   diagonal      1500 non-null   float64
 2   height_left   1500 non-null   float64
 3   height_right  1500 non-null   float64
 4   margin_low    1463 non-null   float64
 5   margin_up     1500 non-null   float64
 6   length        1500 non-null   float64
dtypes:

In [43]:
df_true.describe()

,diagonal,height_left,height_right,margin_low,margin_up,length
count,1000.000000,1000.000000,1000.00000,1000.00000,1000.00000,1000.000000
mean,171.987080,103.949130,103.80865,4.11592,3.05213,113.202430
std,0.300441,0.300231,0.29157,0.31446,0.18634,0.359552
min,171.040000,103.140000,102.82000,2.98000,2.27000,111.760000
25%,171.790000,103.740000,103.61000,3.91000,2.93000,112.950000
50%,171.990000,103.950000,103.81000,4.11000,3.05000,113.205000
75%,172.200000,104.140000,104.00000,4.33000,3.18000,113.460000
max,172.920000,104.860000,104.95000,5.04000,3.74000,114.440000


In [44]:
df_false.describe()

,diagonal,height_left,height_right,margin_low,margin_up,length
count,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000
mean,171.901160,104.190340,104.143620,5.215520,3.350160,111.630640
std,0.306861,0.223758,0.270878,0.549086,0.180498,0.615543
min,171.040000,103.510000,103.430000,3.820000,2.920000,109.490000
25%,171.690000,104.040000,103.950000,4.840000,3.220000,111.200000
50%,171.910000,104.180000,104.160000,5.190000,3.350000,111.630000
75%,172.092500,104.332500,104.320000,5.590000,3.472500,112.030000
max,173.010000,104.880000,104.950000,6.900000,3.910000,113.850000
